In [1]:
# give colab permission to access drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Update these as required
repo_path = '/content/drive/MyDrive/github/subteams/LLMProbing'
odeformer_path = '/content/drive/MyDrive/aisc' # This is because I cloned the odeformer repo into my aisc folder
samples_path = '/content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/samples'
activations_path = '/content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/activations'
probes_path = '/content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes'

In [3]:
import sys
import importlib
sys.path.append(repo_path)
sys.path.append(odeformer_path)

In [4]:
import numpy as np
import os
import pickle

In [5]:
from odeformer.model import SymbolicTransformerRegressor
dstr = SymbolicTransformerRegressor(from_pretrained=True)
model_args = {'beam_size': 10, 'beam_temperature': 0.8}
dstr.set_model_args(model_args)

Found pretrained model at odeformer.pt
Loaded pretrained model


In [6]:
%load_ext autoreload
%autoreload 2

## Random Sample Generation

In [7]:
from src.sample_generation import RandomSamplesGenerator

In [8]:
operators_to_use = "id:1,add:1,mul:1,sin:0.5"
min_dimension = 1
max_dimension = 1
num_samples = 10
seed = 42
sample_descriptor = 'demo'

random_samples_path = f'{samples_path}/demo_random'

In [9]:
# rsg = RandomSamplesGenerator(samples_path=random_samples_path, num_samples=num_samples, operators_to_use=operators_to_use, min_dimension=min_dimension, max_dimension=max_dimension)
rsg = RandomSamplesGenerator()
# TODO: possibly restructure this class to give important parameters to generate_random_samples()
# Then we will be able to rerun generation with different parameters without having to instantiate a new instance of RSG
# Probably what is most useful is to specify samples_path only

In [10]:
rsg.generate_random_samples(random_samples_path, seed=seed, num_samples=num_samples, \
                            operators_to_use=operators_to_use, min_dimension=min_dimension, \
                            max_dimension=max_dimension, sample_descriptor='demo')

Generating demo samples: 100%|██████████| 10/10 [00:01<00:00,  5.39it/s]


[INFO] Data generation complete. Saved 10 demo samples to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/samples/demo_random


In [11]:
# Inspect a random sample to see its keys and some data
random_samples_dir = os.fsencode(random_samples_path)
for random_sample_file in os.listdir(random_samples_dir):
  random_sample_name = os.fsdecode(random_sample_file)
  random_sample_path = os.path.join(random_samples_path, random_sample_name)
  with open(random_sample_path, 'rb') as f:
      random_sample = pickle.load(f)
  print(random_sample.keys())
  print(f"Encoded equation: {random_sample['tree']}")
  print(f"Feature dictionary: {random_sample['feature_dict']}")
  break

dict_keys(['times', 'trajectory', 'tree_encoded', 'skeleton_tree_encoded', 'tree', 'skeleton_tree', 'infos', 'operator_dict', 'feature_dict'])
Encoded equation: -0.9998 * x_0
Feature dictionary: {'log': 0, 'exp': 0, 'tan': 0, 'arctan': 0, 'sin_cos': 0, 'arc_sin_cos': 0, 'pow2': 0, 'pow3': 0, 'inv': 0, 'sqrt': 0}


## Manual Sample Generation

In [12]:
from src.sample_generation import ManualSamplesGenerator

In [13]:
manual_samples_path = f'{samples_path}/demo_manual'
msg = ManualSamplesGenerator(samples_path=manual_samples_path)

In [14]:
# Times array
t_values = np.linspace(1, 10, 50)

# Exponential parameters
c_values = np.linspace(-10,10, 2)
a_values = np.linspace(-10,10, 5)

# Hyperbolic parameters
t0_values = np.linspace(10.1, 100, 5)

In [15]:
# Generate exponential samples
msg.generate_exponential_samples(t_values, c_values, a_values)

Generating exponential samples:   0%|          | 0/10 [00:00<?, ?it/s]

Saving generated samples: 10it [00:00, 75.19it/s]


[INFO] Data generation complete. Saved 10 exponential samples to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/samples/demo_manual


In [16]:
# Inspect an exponential sample to see its keys and some data
manual_samples_dir = os.fsencode(manual_samples_path)
for sample_file in os.listdir(manual_samples_dir):
  sample_name = os.fsdecode(sample_file)
  if "exp" in sample_name:
    exp_sample_path = os.path.join(manual_samples_path, sample_name)
    with open(exp_sample_path, 'rb') as f:
        exp_sample = pickle.load(f)
    print(exp_sample.keys())
    print(exp_sample['expression'])
    print(exp_sample['feature_dict'])
    break

dict_keys(['times', 'trajectory', 'parameters', 'feature_dict', 'expression'])
-10.0 * np.exp(10.0 * t)
{'exponential': 1, 'hyperbolic': 0}


In [17]:
# Generate hyperbolic samples
msg.generate_hyperbolic_samples(t_values, c_values, t0_values)

Generating hyperbolic samples:   0%|          | 0/10 [00:00<?, ?it/s]

Saving generated samples: 10it [00:00, 81.67it/s]


[INFO] Data generation complete. Saved 10 hyperbolic samples to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/samples/demo_manual


In [18]:
# Inspect a hyperbolic sample to see its keys and some data
manual_samples_dir = os.fsencode(manual_samples_path)
for sample_file in os.listdir(manual_samples_dir):
  sample_name = os.fsdecode(sample_file)
  if "hyp" in sample_name:
    hyp_sample_path = os.path.join(manual_samples_path, sample_name)
    with open(hyp_sample_path, 'rb') as f:
        hyp_sample = pickle.load(f)
    print(hyp_sample.keys())
    print(hyp_sample['expression'])
    print(hyp_sample['parameters'])
    print(hyp_sample['feature_dict'])
    break

dict_keys(['times', 'trajectory', 'parameters', 'feature_dict', 'expression'])
-10.0 / (10.1-t)
{'t0': 10.1, 'c': -10.0}
{'exponential': 0, 'hyperbolic': 1}


## Corresponding Activations Extraction

In [19]:
from src.activation_extraction import ActivationsExtractor

In [20]:
random_activations_path = f'{activations_path}/demo_random'
manual_activations_path = f'{activations_path}/demo_manual'

In [21]:
act_extractor = ActivationsExtractor()

In [22]:
act_extractor.extract_activations(dstr, random_samples_path, random_activations_path, layers_to_extract=['ffn'])

Extracting Activations: 100%|██████████| 10/10 [00:33<00:00,  3.38s/it]


[INFO] Activation extraction complete. Activations saved to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/activations/demo_random


In [23]:
act_extractor.extract_activations(dstr, manual_samples_path, manual_activations_path, layers_to_extract=['ffn'])

Extracting Activations: 100%|██████████| 20/20 [01:35<00:00,  4.77s/it]


[INFO] Activation extraction complete. Activations saved to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/activations/demo_manual


In [24]:
# Inspect a random activation and display its keys and some values
random_acts_dir = os.fsencode(random_activations_path)
for acts_file in os.listdir(random_acts_dir):
  acts_name = os.fsdecode(acts_file)
  random_acts_path = os.path.join(random_activations_path, acts_name)
  with open(random_acts_path, 'rb') as f:
      random_acts = pickle.load(f)
  print(random_acts.keys())
  print(random_acts['feature_dict'])
  print(random_acts['r2_score'])
  print(random_acts['expression'])
  print(random_acts['pred_expression'])
  break

dict_keys(['encoder', 'decoder', 'operator_dict', 'feature_dict', 'r2_score', 'pred_expression', 'expression'])
{'log': 0, 'exp': 0, 'tan': 0, 'arctan': 0, 'sin_cos': 0, 'arc_sin_cos': 0, 'pow2': 0, 'pow3': 0, 'inv': 0, 'sqrt': 0}
0.9999999755587363
-0.9998 * x_0
x_0' = -1.0000000000000000 * x_0



In [25]:
# Inspect a manual activation and display its keys and some values
manual_acts_dir = os.fsencode(manual_activations_path)
for acts_file in os.listdir(manual_acts_dir):
  acts_name = os.fsdecode(acts_file)
  manual_acts_path = os.path.join(manual_activations_path, acts_name)
  with open(manual_acts_path, 'rb') as f:
      manual_acts = pickle.load(f)
  print(manual_acts.keys())
  print(manual_acts['feature_dict'])
  print(manual_acts['r2_score'])
  print(manual_acts['expression'])
  print(manual_acts['pred_expression'])
  break

dict_keys(['encoder', 'decoder', 'feature_dict', 'r2_score', 'pred_expression', 'expression'])
{'exponential': 1, 'hyperbolic': 0}
-0.0374609619209898
-10.0 * np.exp(5.0 * t)
x_0' = 8.5950 * x_0 + 0.2052 * x_0 * (-22.7200 + 0.1314 * (0.0555 + -0.e-4 * x_0)**-1)



## Probe Training

In [26]:
import experiments

In [27]:
dir(experiments)
# Current experiment functions supported:
#   - separability_testing

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'run_experiment',
 'separability_testing']

In [28]:
# from experiments.run_experiment import separability_testing

In [29]:
target_feature = 'exponential'
activations_path = manual_activations_path
demo_probes_path = f'{probes_path}/demo_exp'
lr = 0.01
num_epochs = 1
num_repeats = 2
layers = [idx for idx in range(4, 16)]

In [30]:
demo_expt_results = experiments.separability_testing(target_feature=target_feature, activations_path=manual_activations_path, \
                     probes_path=demo_probes_path, \
                     lr=lr, num_epochs=num_epochs, \
                     layers=layers, num_repeats=num_repeats)

Training LR Probe: 100%|██████████| 1/1 [00:00<00:00,  5.66it/s]



Epoch 1 (Final): Loss 0.14097990095615387, Accuracy 0.25

Probe trained on layer 4:
Test Set: Loss 6.704820013684298, Accuracy 0.5
Saved state dictionary to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes/demo_exp/probe_exponential_4_0.pt


Training LR Probe: 100%|██████████| 1/1 [00:00<00:00,  4.58it/s]



Epoch 1 (Final): Loss 0.11276042461395264, Accuracy 0.3125

Probe trained on layer 4:
Test Set: Loss 6.717358599098406, Accuracy 0.5
Saved state dictionary to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes/demo_exp/probe_exponential_4_1.pt


Training LR Probe: 100%|██████████| 1/1 [00:00<00:00,  6.55it/s]


...


Epoch 1 (Final): Loss 0.05744853988289833, Accuracy 0.5625

Probe trained on layer 14:
Test Set: Loss 0.7286160010844469, Accuracy 0.5
Saved state dictionary to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes/demo_exp/probe_exponential_14_1.pt


Training LR Probe: 100%|██████████| 1/1 [00:00<00:00,  5.59it/s]



Epoch 1 (Final): Loss 0.0690440833568573, Accuracy 0.4375

Probe trained on layer 15:
Test Set: Loss 1.1409718990325928, Accuracy 0.5
Saved state dictionary to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes/demo_exp/probe_exponential_15_0.pt


Training LR Probe: 100%|██████████| 1/1 [00:00<00:00,  5.77it/s]



Epoch 1 (Final): Loss 0.038657963275909424, Accuracy 0.8125

Probe trained on layer 15:
Test Set: Loss 0.1157916821539402, Accuracy 1.0
Saved state dictionary to /content/drive/MyDrive/github/subteams/LLMProbing/local_experiment_data/probes/demo_exp/probe_exponential_15_1.pt


In [31]:
demo_expt_results

,layer,run,test_loss,test_accuracy,test_fail_ids,final_train_loss,final_train_accuracy,final_val_loss,final_val_accuracy
0,4,0,6.704820,0.5,[hyp_1],0.140980,0.2500,1.290589e-06,1.0
1,4,1,6.717359,0.5,[hyp_1],0.112760,0.3125,1.771318e-07,1.0
2,5,0,0.921291,0.5,[hyp_1],0.059685,0.1875,6.766988e-02,1.0
3,5,1,2.003600,0.5,[exp_8],0.040516,0.5625,4.165272e+00,0.0
4,6,0,0.462544,1.0,[],0.047725,0.5625,4.247469e-01,1.0
5,6,1,0.537468,1.0,[],0.049375,0.1875,7.081534e-01,0.5
6,7,0,0.790847,0.5,[exp_8],0.045085,0.4375,1.326330e+00,0.0
7,7,1,0.825522,0.5,[exp_8],0.045070,0.4375,1.396905e+00,0.0
8,8,0,0.359831,1.0,[],0.046766,0.4375,7.004579e-01,0.5
9,8,1,0.598939,0.5,[exp_8],0.042862,0.4375,1.504264e+00,0.0


In [32]:
# TODO: view and summarise experiment results
import experiments.utils as expt_utils

expt_utils.summarise_experiment(demo_expt_results)

,layer,accuracy_mean,accuracy_std,loss_mean,loss_std
0,4,0.50,0.000000,6.711089,0.008866
1,5,0.50,0.000000,1.462445,0.765308
2,6,1.00,0.000000,0.500006,0.052980
3,7,0.50,0.000000,0.808185,0.024519
4,8,0.75,0.353553,0.479385,0.169075
5,9,1.00,0.000000,0.264468,0.036178
6,10,1.00,0.000000,0.218595,0.094663
7,11,0.75,0.353553,0.465456,0.013426
8,12,0.75,0.353553,0.536883,0.206588
9,13,0.50,0.000000,0.686777,0.100368


## Probe Loading and Evaluation

In [33]:
# TODO: write extra loading functionality for running an experiment using pretrained
# TODO: extend failure detection functionality